In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

output_dir = "result/"

# 결과 저장 폴더 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load data from the CSV files
file_paths = {
    "like": "like.csv",
    "num_comments": "num_comments.csv",
    "time": "time.csv",
    "view": "view.csv",
    "like_per_view": "like_per_view.csv",
    "comments_per_view": "comments_per_view.csv",
    "view_per_one": "view_per_one.csv"
}


# 관심 있는 카테고리 번호 설정 (예: 카테고리 1, 2)
categories = [1,2,10,15,17,19,20,22,23,24,25,26,27,28,29]

# 각 데이터셋에 대해 반복 작업 수행
for key, path in file_paths.items():
    df = pd.read_csv(path, index_col=0)
    df = df.dropna(how='any')
    try:
        df.index = pd.to_datetime(df.index, format='%Y-%m')
    except:
        df.index = pd.to_datetime(df.index, format='%y-%b')
    
    df = df.sort_index()

    for i, cat in enumerate(categories):
        X = pd.DataFrame({'year': df.index.year, 'month': df.index.month})
        y = df[str(cat)].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Linear Regression 모델
        lin_model = LinearRegression()
        lin_model.fit(X_train, y_train)
        y_pred_lin = lin_model.predict(X_test)
        
        # Random Forest 모델
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        y_pred_rf = rf_model.predict(X_test)
        
        # 두 모델의 예측값 평균
        y_pred_ensemble = (y_pred_lin + y_pred_rf) / 2
        
        mse = mean_squared_error(y_test, y_pred_ensemble)
        print(f'{key} 데이터셋 카테고리 {cat} - 앙상블 모델 평균 제곱 오차: {mse}')
        
        # 2024년 5월부터 9월까지의 예측
        future_dates = pd.date_range(start='2024-05-01', end='2024-09-01', freq='MS')
        future = pd.DataFrame({
            'year': future_dates.year,
            'month': future_dates.month
        })
        future_pred_lin = lin_model.predict(future)
        future_pred_rf = rf_model.predict(future)
        future_pred_ensemble = (future_pred_lin + future_pred_rf) / 2
        
        # 2024년 4월의 실제 값을 포함하여 예측 데이터와 연결
        extended_dates = df.index.append(future_dates)
        extended_values = np.concatenate([y, future_pred_ensemble])
        
        # Linear Regression을 사용하여 전체 데이터에 대한 추세선 계산
        trendline = lin_model.predict(pd.DataFrame({
            'year': extended_dates.year,
            'month': extended_dates.month
        }))
        
        # 그래프 그리기
        plt.figure(figsize=(10, 6))
        
        # 원본 데이터
        plt.plot(df.index, y, label='원본 데이터', color='blue')
        
        # 추세선
        plt.plot(extended_dates, trendline, label='추세선', color='green', linestyle='--')
        
        # 예측 데이터와 4월 데이터를 연결하는 점선
        plt.plot(future_dates, future_pred_ensemble, 'ro', label=f'카테고리 {cat} 예측값 (5월-9월)')
        plt.plot([df.index[-1], future_dates[0]], [y[-1], future_pred_ensemble[0]], 'r--')
        plt.plot(future_dates, future_pred_ensemble, 'r--')

        plt.title(f'{key.capitalize()} 데이터 예측 (카테고리 {cat})')
        plt.xlabel('시간')
        plt.ylabel(key.capitalize())
        plt.legend()
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.gcf().autofmt_xdate()
        plt.grid(True)
        
        # 그래프 저장
        graph_filename = os.path.join(output_dir, f'{cat}_{key}_graph.png')
        plt.savefig(graph_filename)
        plt.close()
        
        # 데이터를 CSV로 내보내기
        output_df = pd.DataFrame({
            'Date': extended_dates,
            'Original': np.concatenate([y, [np.nan] * len(future_dates)]),  # 실제 값
            'Trendline': trendline,  # 추세선 값
            'Prediction': np.concatenate([[np.nan] * (len(df.index)-1), [y[-1]], future_pred_ensemble])  # 예측 값 (4월 실 데이터 포함)
        })
        output_file = os.path.join(output_dir, f'{key}_category_{cat}_forecast.csv')
        output_df.to_csv(output_file, index=False)
